## Library Installation

Ensure that the necessary libraries are installed before running the notebook.

In [1]:
# Import necessary libraries
%pip install openpyxl
from openpyxl import load_workbook
import pandas as pd
import os
import re
import itertools

print("Skeleton setup complete!")


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Skeleton setup complete!


## Variable Declaration

Set the variables for file paths, sheet names, and other configurations. Update these variables for each specific project.

In [2]:
# Path to the Excel file (change this for each project)
excel_file_path = '/workspaces/Finetwork-Automation/inbound/Metabase.xlsx'
csv_file_path = '/workspaces/Finetwork-Automation/inbound/Informe de métricas históricas.csv'

# Sheet names for different categories
sheet_active = 'ACTIVOS'
sheet_canceled = 'CANCELADOS'
sheet_pending = 'PTE DE FIRMA'
sheet_value = 'TOP VALOR'
sheet_packs = 'TOP PACKS'
sheet_services = 'TOP SERVICIOS'

# Range to read (change this for each project)
start_row = 8
end_row = 150
usecols = 'A:F'

print("Variables defined correctly!")

Variables defined correctly!


## Verify Columns in 'Active' Sheet.

Verify the number of columns in the "Active" sheet to ensure the range is within bounds. We will use the range in all three sheets

In [ ]:
# Function to verify the number of columns
def verify_columns(file_path, sheet_name):
    workbook = load_workbook(filename=file_path, data_only=True)
    sheet = workbook[sheet_name]
    max_column = sheet.max_column
    return max_column

# Check the number of columns in the 'Active' sheet
max_column_active = verify_columns(excel_file_path, 'ACTIVOS')
print(f"Max column in 'Active' sheet: {max_column_active}")

# Check if the number of columns matches the expected range
expected_columns = 32  # Columns from A to AF (inclusive)
if max_column_active < expected_columns:
    usecols = f"A:{chr(64+max_column_active)}"
    print(f"Adjusted usecols to: {usecols}")
else:
    print(f"Using default usecols: {usecols}")

## Extract Data from all six Sheets

Extract data from all three Sheets within the specified range and convert it directly to a DataFrame.

In [3]:
def load_sheet_as_dataframe(file_path, sheet_name, start_row, end_row, usecols):
    # Load data from the specified sheet and range into a DataFrame
    df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=usecols, skiprows=start_row-1, nrows=end_row-start_row+1)
    print(f"Data from '{sheet_name}' sheet loaded successfully.")
    return df

# Extract data from 'Active' sheet
active_df = load_sheet_as_dataframe(excel_file_path, 'ACTIVOS', start_row, end_row, usecols)
canceled_df = load_sheet_as_dataframe(excel_file_path, 'CANCELADOS', start_row, end_row, usecols)
pending_signature_df = load_sheet_as_dataframe(excel_file_path, 'PTE FIRMA', start_row, end_row, usecols)
value_df = load_sheet_as_dataframe(excel_file_path, 'TOP VALOR', start_row, end_row, usecols)
packs_df = load_sheet_as_dataframe(excel_file_path, 'TOP PACKS', start_row, end_row, usecols)
services_df = load_sheet_as_dataframe(excel_file_path, 'TOP SERVICIOS', start_row, end_row, usecols)

# Display the DataFrames
print("Active DataFrame:")
display(active_df.head(99))

print("Canceled DataFrame:")
display(canceled_df.head())

print("Pending Signature DataFrame:")
display(pending_signature_df.head())

print("value DataFrame:")
display(value_df.head())

print("Packs DataFrame:")
display(packs_df.head())

print("Services DataFrame:")
display(services_df.head())


Data from 'ACTIVOS' sheet loaded successfully.
Data from 'CANCELADOS' sheet loaded successfully.
Data from 'PTE FIRMA' sheet loaded successfully.
Data from 'TOP VALOR' sheet loaded successfully.
Data from 'TOP PACKS' sheet loaded successfully.
Data from 'TOP SERVICIOS' sheet loaded successfully.
Active DataFrame:


,Inbound Telec.Orig.Sevilla,187,715,785,625,2312
0,albaaraujo@originaltelecom.es,NaN,NaN,25.0,20.0,45
1,albertocanto@originaltelecom.es,NaN,19.0,11.0,12.0,42
2,andrea.deltell@finetwork.com,NaN,1.0,NaN,1.0,2
3,azahara.garcia@originaltelecom.es,29.0,NaN,NaN,NaN,29
4,beatriz.gilperez@grupokonecta.es,NaN,1.0,NaN,NaN,1
...,...,...,...,...,...,...
88,sara.elkhelyfy@originaltelecom.es,NaN,31.0,41.0,23.0,95
89,sergio.vazquez@originaltelecom.es,NaN,19.0,16.0,23.0,58
90,yicel.patricia@originaltelecom.es,12.0,16.0,6.0,NaN,34
91,yzabelly.gomes@originaltelecom.es,24.0,NaN,NaN,NaN,24


Canceled DataFrame:


,Inbound Telec.Orig.Sevilla,16,62,40,40.1,158
0,alberto.valle@finetwork.com,NaN,1.0,NaN,NaN,1
1,albertocanto@originaltelecom.es,NaN,1.0,NaN,1.0,2
2,azahara.garcia@originaltelecom.es,1.0,NaN,NaN,NaN,1
3,carolinafuentes@originaltelecom.es,NaN,1.0,1.0,4.0,6
4,david.molero@originaltelecom.es,NaN,9.0,1.0,2.0,12


Pending Signature DataFrame:


...

,Inbound Telec.Orig.Sevilla,83,378,399,310,1170
0,albaaraujo@originaltelecom.es,NaN,NaN,10.0,8.0,18
1,albertocanto@originaltelecom.es,NaN,13.0,5.0,7.0,25
2,azahara.garcia@originaltelecom.es,12.0,NaN,NaN,NaN,12
3,beatriz.gomez@originaltelecom.es,NaN,NaN,5.0,8.0,13
4,carolinafuentes@originaltelecom.es,NaN,12.0,8.0,15.0,35


Packs DataFrame:


,Inbound Telec.Orig.Sevilla,111,465,515,360,1451
0,albaaraujo@originaltelecom.es,NaN,NaN,13.0,17.0,30
1,albertocanto@originaltelecom.es,NaN,11.0,10.0,7.0,28
2,azahara.garcia@originaltelecom.es,18.0,NaN,NaN,NaN,18
3,beatriz.gomez@originaltelecom.es,NaN,NaN,9.0,13.0,22
4,carolinafuentes@originaltelecom.es,NaN,13.0,15.0,12.0,40


Services DataFrame:


,Inbound Telec.Orig.Sevilla,187,715,785,625,2312
0,albaaraujo@originaltelecom.es,NaN,NaN,25.0,20.0,45
1,albertocanto@originaltelecom.es,NaN,19.0,11.0,12.0,42
2,andrea.deltell@finetwork.com,NaN,1.0,NaN,1.0,2
3,azahara.garcia@originaltelecom.es,29.0,NaN,NaN,NaN,29
4,beatriz.gilperez@grupokonecta.es,NaN,1.0,NaN,NaN,1


## Replace NaN with 0

Replace all NaN values in the DataFrames with 0 to facilitate further transformations.

In [4]:
def replace_nan_with_zero(df):
    """
    Replace all NaN values in the DataFrame with 0.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to process.
    
    Returns:
    pd.DataFrame: The processed DataFrame with NaN replaced by 0.
    """
    df = df.fillna(0)
    print("Replaced NaN with 0.")
    return df

active_df = replace_nan_with_zero(active_df)
canceled_df = replace_nan_with_zero(canceled_df)
pending_signature_df = replace_nan_with_zero(pending_signature_df)
value_df = replace_nan_with_zero(value_df)
packs_df = replace_nan_with_zero(packs_df)
services_df = replace_nan_with_zero(services_df)



# Display the transformed DataFrames
print("Active DataFrame after replacing NaN:")
display(active_df.head(88))

print("Canceled DataFrame after replacing NaN:")
display(canceled_df.head())

print("Pending Signature DataFrame after replacing NaN:")
display(pending_signature_df.head())

print("Value DataFrame after replacing NaN:")
display(value_df.head())

print("Packs DataFrame after replacing NaN:")
display(packs_df.head())

print("Services DataFrame after replacing NaN:")
display(services_df.head())

Replaced NaN with 0.
Replaced NaN with 0.
Replaced NaN with 0.
Replaced NaN with 0.
Replaced NaN with 0.
Replaced NaN with 0.
Active DataFrame after replacing NaN:


,Inbound Telec.Orig.Sevilla,187,715,785,625,2312
0,albaaraujo@originaltelecom.es,0.0,0.0,25.0,20.0,45
1,albertocanto@originaltelecom.es,0.0,19.0,11.0,12.0,42
2,andrea.deltell@finetwork.com,0.0,1.0,0.0,1.0,2
3,azahara.garcia@originaltelecom.es,29.0,0.0,0.0,0.0,29
4,beatriz.gilperez@grupokonecta.es,0.0,1.0,0.0,0.0,1
...,...,...,...,...,...,...
83,[PACK] Fibra 600 Mbps y 10 GB.,0.0,2.0,0.0,0.0,2
84,rcardenasg@konecta-group.com,0.0,0.0,1.0,4.0,5
85,[PACK] Fibra 600 Mbps y 55 GB.,0.0,0.0,1.0,0.0,1
86,Tarifa de 10 GB y llamadas ilimitadas.,0.0,0.0,0.0,1.0,1


Canceled DataFrame after replacing NaN:


,Inbound Telec.Orig.Sevilla,16,62,40,40.1,158
0,alberto.valle@finetwork.com,0.0,1.0,0.0,0.0,1
1,albertocanto@originaltelecom.es,0.0,1.0,0.0,1.0,2
2,azahara.garcia@originaltelecom.es,1.0,0.0,0.0,0.0,1
3,carolinafuentes@originaltelecom.es,0.0,1.0,1.0,4.0,6
4,david.molero@originaltelecom.es,0.0,9.0,1.0,2.0,12


Pending Signature DataFrame after replacing NaN:


...

,Inbound Telec.Orig.Sevilla,83,378,399,310,1170
0,albaaraujo@originaltelecom.es,0.0,0.0,10.0,8.0,18
1,albertocanto@originaltelecom.es,0.0,13.0,5.0,7.0,25
2,azahara.garcia@originaltelecom.es,12.0,0.0,0.0,0.0,12
3,beatriz.gomez@originaltelecom.es,0.0,0.0,5.0,8.0,13
4,carolinafuentes@originaltelecom.es,0.0,12.0,8.0,15.0,35


Packs DataFrame after replacing NaN:


,Inbound Telec.Orig.Sevilla,111,465,515,360,1451
0,albaaraujo@originaltelecom.es,0.0,0.0,13.0,17.0,30
1,albertocanto@originaltelecom.es,0.0,11.0,10.0,7.0,28
2,azahara.garcia@originaltelecom.es,18.0,0.0,0.0,0.0,18
3,beatriz.gomez@originaltelecom.es,0.0,0.0,9.0,13.0,22
4,carolinafuentes@originaltelecom.es,0.0,13.0,15.0,12.0,40


Services DataFrame after replacing NaN:


,Inbound Telec.Orig.Sevilla,187,715,785,625,2312
0,albaaraujo@originaltelecom.es,0.0,0.0,25.0,20.0,45
1,albertocanto@originaltelecom.es,0.0,19.0,11.0,12.0,42
2,andrea.deltell@finetwork.com,0.0,1.0,0.0,1.0,2
3,azahara.garcia@originaltelecom.es,29.0,0.0,0.0,0.0,29
4,beatriz.gilperez@grupokonecta.es,0.0,1.0,0.0,0.0,1


## Load the list of agents

In [5]:
# Load the list of agents
agents_df = pd.read_excel(excel_file_path, sheet_name='AGENTES', usecols='A')
agents_list = agents_df.iloc[:, 0].tolist()
print("Agents list loaded successfully!")
print(agents_list)

Agents list loaded successfully!
['albaaraujo@originaltelecom.es', 'albertocanto@originaltelecom.es', 'albertosanchez@originaltelecom.es', 'antonio.reina@originaltelecom.es', 'azahara.garcia@originaltelecom.es', 'beatriz.gomez@originaltelecom.es', 'maricarmen.cornejo@originaltelecom.es', 'carolinafuentes@originaltelecom.es', 'cesar.arnaldo@originaltelecom.es', 'david.molero@originaltelecom.es', 'elenaborrero@originaltelecom.es', 'estefania.panea@originaltelecom.es', 'francisco.perdomo@originaltelecom.es', 'gonzalofalcon@originaltelecom.es', 'guillermo.hurtado@originaltelecom.es', 'irati.izaguirre@originaltelecom.es', 'ivan.barroso@originaltelecom.es', 'laura.eguens@originaltelecom.es', 'lailasetati@originaltelecom.es', 'leonor.lopez@originaltelecom.es', 'dolores.cortes@originaltelecom.es', 'manuelcabra@originaltelecom.es', 'manuelvaldes@originaltelecom.es', 'manuelventura@originaltelecom.es', 'mariangeles.bueso@originaltelecom.es', 'mariaarroyo@originaltelecom.es', 'maria.torres@origin

## Verify and Complete Data

Verify that all agents are present in each DataFrame. If an agent is missing, add a row with zeros for that agent.

In [6]:
def ensure_all_agents(df, agents_list):
    """
    Ensure all agents are present in the DataFrame. Add missing agents with zero values and remove agents not in the list.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to check and update.
    agents_list (list): The list of all agents.
    
    Returns:
    pd.DataFrame: The updated DataFrame with all agents.
    """
    # Get the list of agents in the DataFrame
    existing_agents = df.iloc[:, 0].tolist()
    
    # Find missing agents
    missing_agents = [agent for agent in agents_list if agent not in existing_agents]
    
    # Add rows for missing agents with zero values
    for agent in missing_agents:
        zero_row = pd.DataFrame([[agent] + [0] * (df.shape[1] - 1)], columns=df.columns)
        df = pd.concat([df, zero_row], ignore_index=True)
    
    # Remove agents not in the agents list
    df = df[df.iloc[:, 0].isin(agents_list)]
    
    print(f"Added {len(missing_agents)} missing agents and removed {len(existing_agents) - len(df)} agents not in the list.")
    return df

# Apply the function to each DataFrame
active_df = ensure_all_agents(active_df, agents_list)
canceled_df = ensure_all_agents(canceled_df, agents_list)
pending_signature_df = ensure_all_agents(pending_signature_df, agents_list)
value_df = ensure_all_agents(value_df, agents_list)
packs_df = ensure_all_agents(packs_df, agents_list)
services_df = ensure_all_agents(services_df, agents_list)

# Display the updated DataFrames
print("Active DataFrame after ensuring all agents:")
display(active_df.head(88))

print("Canceled DataFrame after ensuring all agents:")
display(canceled_df.head())

print("Pending Signature DataFrame after ensuring all agents:")
display(pending_signature_df.head())

print("Value DataFrame after ensuring all agents:")
display(value_df.head())

print("Packs DataFrame after ensuring all agents:")
display(packs_df.head())

print("Services DataFrame after ensuring all agents:")
display(services_df.head())

Added 7 missing agents and removed 50 agents not in the list.
Added 14 missing agents and removed 8 agents not in the list.
Added 21 missing agents and removed -17 agents not in the list.
Added 7 missing agents and removed 39 agents not in the list.
Added 7 missing agents and removed 28 agents not in the list.
Added 7 missing agents and removed 50 agents not in the list.
Active DataFrame after ensuring all agents:


,Inbound Telec.Orig.Sevilla,187,715,785,625,2312
0,albaaraujo@originaltelecom.es,0.0,0.0,25.0,20.0,45
1,albertocanto@originaltelecom.es,0.0,19.0,11.0,12.0,42
3,azahara.garcia@originaltelecom.es,29.0,0.0,0.0,0.0,29
5,beatriz.gomez@originaltelecom.es,0.0,0.0,19.0,15.0,34
6,carolinafuentes@originaltelecom.es,0.0,31.0,17.0,24.0,72
7,cesar.arnaldo@originaltelecom.es,21.0,0.0,0.0,0.0,21
8,david.molero@originaltelecom.es,0.0,52.0,21.0,19.0,92
11,dolores.cortes@originaltelecom.es,0.0,26.0,25.0,20.0,71
12,elenaborrero@originaltelecom.es,0.0,17.0,36.0,25.0,78
13,estefania.panea@originaltelecom.es,0.0,0.0,23.0,23.0,46


Canceled DataFrame after ensuring all agents:


,Inbound Telec.Orig.Sevilla,16,62,40,40.1,158
1,albertocanto@originaltelecom.es,0.0,1.0,0.0,1.0,2
2,azahara.garcia@originaltelecom.es,1.0,0.0,0.0,0.0,1
3,carolinafuentes@originaltelecom.es,0.0,1.0,1.0,4.0,6
4,david.molero@originaltelecom.es,0.0,9.0,1.0,2.0,12
6,dolores.cortes@originaltelecom.es,0.0,4.0,1.0,0.0,5


Pending Signature DataFrame after ensuring all agents:


...

,Inbound Telec.Orig.Sevilla,83,378,399,310,1170
0,albaaraujo@originaltelecom.es,0.0,0.0,10.0,8.0,18
1,albertocanto@originaltelecom.es,0.0,13.0,5.0,7.0,25
2,azahara.garcia@originaltelecom.es,12.0,0.0,0.0,0.0,12
3,beatriz.gomez@originaltelecom.es,0.0,0.0,5.0,8.0,13
4,carolinafuentes@originaltelecom.es,0.0,12.0,8.0,15.0,35


Packs DataFrame after ensuring all agents:


,Inbound Telec.Orig.Sevilla,111,465,515,360,1451
0,albaaraujo@originaltelecom.es,0.0,0.0,13.0,17.0,30
1,albertocanto@originaltelecom.es,0.0,11.0,10.0,7.0,28
2,azahara.garcia@originaltelecom.es,18.0,0.0,0.0,0.0,18
3,beatriz.gomez@originaltelecom.es,0.0,0.0,9.0,13.0,22
4,carolinafuentes@originaltelecom.es,0.0,13.0,15.0,12.0,40


Services DataFrame after ensuring all agents:


,Inbound Telec.Orig.Sevilla,187,715,785,625,2312
0,albaaraujo@originaltelecom.es,0.0,0.0,25.0,20.0,45
1,albertocanto@originaltelecom.es,0.0,19.0,11.0,12.0,42
3,azahara.garcia@originaltelecom.es,29.0,0.0,0.0,0.0,29
5,beatriz.gomez@originaltelecom.es,0.0,0.0,19.0,15.0,34
6,carolinafuentes@originaltelecom.es,0.0,31.0,17.0,24.0,72


## Assign Values to Emails

Assign numerical values to each email and add them as a new column in the DataFrames.

In [7]:
# Dictionary mapping emails to their respective values
email_values = {
    'albaaraujo@originaltelecom.es': 1,
    'albertocanto@originaltelecom.es': 2,
    'albertosanchez@originaltelecom.es': 3,
    'antonio.reina@originaltelecom.es': 4,
    'azahara.garcia@originaltelecom.es': 5,
    'beatriz.gomez@originaltelecom.es': 6,
    'maricarmen.cornejo@originaltelecom.es': 7,
    'carolinafuentes@originaltelecom.es': 8,
    'cesar.arnaldo@originaltelecom.es': 9,
    'david.molero@originaltelecom.es': 10,
    'elenaborrero@originaltelecom.es': 11,
    'estefania.panea@originaltelecom.es': 12,
    'francisco.perdomo@originaltelecom.es': 13,
    'gonzalofalcon@originaltelecom.es': 14,
    'guillermo.hurtado@originaltelecom.es': 15,
    'irati.izaguirre@originaltelecom.es': 16,
    'ivan.barroso@originaltelecom.es': 17,
    'laura.eguens@originaltelecom.es': 18,
    'lailasetati@originaltelecom.es': 19,
    'leonor.lopez@originaltelecom.es': 20,
    'dolores.cortes@originaltelecom.es': 21,
    'manuelcabra@originaltelecom.es': 22,
    'manuelvaldes@originaltelecom.es': 23,
    'manuelventura@originaltelecom.es': 24,
    'mariangeles.bueso@originaltelecom.es': 25,
    'mariaarroyo@originaltelecom.es': 26,
    'maria.torres@originaltelecom.es': 27,
    'mario.infante@originaltelecom.es': 28,
    'marta.dorado@originaltelecom.es': 29,
    'mauricio.pozo@originaltelecom.es': 30,
    'miguel.segura@originaltelecom.es': 31,
    'miriam.rodriguez@originaltelecom.es': 32,
    'mar.marchena@originaltelecom.es': 33,
    'natividad.sanchez@originaltelecom.es': 34,
    'oscar.rivilla@originaltelecom.es': 35,
    'patricia.rios@originaltelecom.es': 36,
    'paulavilla@originaltelecom.es': 37,
    'pedro.manzanero@originaltelecom.es': 38,
    'sara.elkhelyfy@originaltelecom.es': 39,
    'sergio.vazquez@originaltelecom.es': 40,
    'tania.arenas@originaltelecom.es': 41,
    'yicel.patricia@originaltelecom.es': 42,
    'yzabelly.gomes@originaltelecom.es': 43
}

# Add a new column to each DataFrame with the email values
def add_email_values(df, email_values):
    df['email_value'] = df.iloc[:, 0].map(email_values)
    return df

# Apply the function to each DataFrame
active_df = add_email_values(active_df, email_values)
canceled_df = add_email_values(canceled_df, email_values)
pending_signature_df = add_email_values(pending_signature_df, email_values)
value_df = add_email_values(value_df, email_values)
packs_df = add_email_values(packs_df, email_values)
services_df = add_email_values(services_df, email_values)

# Display the updated DataFrames with the new 'email_value' column
print("Active DataFrame with email values:")
display(active_df.head())

print("Canceled DataFrame with email values:")
display(canceled_df.head())

print("Pending Signature DataFrame with email values:")
display(pending_signature_df.head())

print("Value DataFrame with email values:")
display(value_df.head())

print("Packs DataFrame with email values:")
display(packs_df.head())

print("Services DataFrame with email values:")
display(services_df.head())

Active DataFrame with email values:


,Inbound Telec.Orig.Sevilla,187,715,785,625,2312,email_value
0,albaaraujo@originaltelecom.es,0.0,0.0,25.0,20.0,45,1
1,albertocanto@originaltelecom.es,0.0,19.0,11.0,12.0,42,2
3,azahara.garcia@originaltelecom.es,29.0,0.0,0.0,0.0,29,5
5,beatriz.gomez@originaltelecom.es,0.0,0.0,19.0,15.0,34,6
6,carolinafuentes@originaltelecom.es,0.0,31.0,17.0,24.0,72,8


Canceled DataFrame with email values:


,Inbound Telec.Orig.Sevilla,16,62,40,40.1,158,email_value
1,albertocanto@originaltelecom.es,0.0,1.0,0.0,1.0,2,2
2,azahara.garcia@originaltelecom.es,1.0,0.0,0.0,0.0,1,5
3,carolinafuentes@originaltelecom.es,0.0,1.0,1.0,4.0,6,8
4,david.molero@originaltelecom.es,0.0,9.0,1.0,2.0,12,10
6,dolores.cortes@originaltelecom.es,0.0,4.0,1.0,0.0,5,21


Pending Signature DataFrame with email values:


...

,Inbound Telec.Orig.Sevilla,83,378,399,310,1170,email_value
0,albaaraujo@originaltelecom.es,0.0,0.0,10.0,8.0,18,1
1,albertocanto@originaltelecom.es,0.0,13.0,5.0,7.0,25,2
2,azahara.garcia@originaltelecom.es,12.0,0.0,0.0,0.0,12,5
3,beatriz.gomez@originaltelecom.es,0.0,0.0,5.0,8.0,13,6
4,carolinafuentes@originaltelecom.es,0.0,12.0,8.0,15.0,35,8


Packs DataFrame with email values:


,Inbound Telec.Orig.Sevilla,111,465,515,360,1451,email_value
0,albaaraujo@originaltelecom.es,0.0,0.0,13.0,17.0,30,1
1,albertocanto@originaltelecom.es,0.0,11.0,10.0,7.0,28,2
2,azahara.garcia@originaltelecom.es,18.0,0.0,0.0,0.0,18,5
3,beatriz.gomez@originaltelecom.es,0.0,0.0,9.0,13.0,22,6
4,carolinafuentes@originaltelecom.es,0.0,13.0,15.0,12.0,40,8


Services DataFrame with email values:


,Inbound Telec.Orig.Sevilla,187,715,785,625,2312,email_value
0,albaaraujo@originaltelecom.es,0.0,0.0,25.0,20.0,45,1
1,albertocanto@originaltelecom.es,0.0,19.0,11.0,12.0,42,2
3,azahara.garcia@originaltelecom.es,29.0,0.0,0.0,0.0,29,5
5,beatriz.gomez@originaltelecom.es,0.0,0.0,19.0,15.0,34,6
6,carolinafuentes@originaltelecom.es,0.0,31.0,17.0,24.0,72,8


## Sort DataFrames by Email Values

Sort the DataFrames based on the numerical values assigned to the emails.

In [8]:
# Sort each DataFrame by the 'email_value' column
def sort_by_email_value(df):
    df = df.sort_values(by='email_value')
    return df

# Apply the sorting function to each DataFrame
active_df = sort_by_email_value(active_df)
canceled_df = sort_by_email_value(canceled_df)
pending_signature_df = sort_by_email_value(pending_signature_df)
value_df = sort_by_email_value(value_df)
packs_df = sort_by_email_value(packs_df)
services_df = sort_by_email_value(services_df)

# Display the sorted DataFrames
print("Sorted Active DataFrame:")
display(active_df.head()) 

print("Sorted Canceled DataFrame:")
display(canceled_df.head())

print("Sorted Pending Signature DataFrame:")
display(pending_signature_df.head())

print("Sorted Canceled DataFrame:")
display(value_df.head())

print("Sorted Pending Signature DataFrame:")
display(packs_df.head())

print("Sorted Pending Signature DataFrame:")
display(services_df.head())

Sorted Active DataFrame:


,Inbound Telec.Orig.Sevilla,187,715,785,625,2312,email_value
0,albaaraujo@originaltelecom.es,0.0,0.0,25.0,20.0,45,1
1,albertocanto@originaltelecom.es,0.0,19.0,11.0,12.0,42,2
93,albertosanchez@originaltelecom.es,0.0,0.0,0.0,0.0,0,3
94,antonio.reina@originaltelecom.es,0.0,0.0,0.0,0.0,0,4
3,azahara.garcia@originaltelecom.es,29.0,0.0,0.0,0.0,29,5


Sorted Canceled DataFrame:


,Inbound Telec.Orig.Sevilla,16,62,40,40.1,158,email_value
51,albaaraujo@originaltelecom.es,0.0,0.0,0.0,0.0,0,1
1,albertocanto@originaltelecom.es,0.0,1.0,0.0,1.0,2,2
52,albertosanchez@originaltelecom.es,0.0,0.0,0.0,0.0,0,3
53,antonio.reina@originaltelecom.es,0.0,0.0,0.0,0.0,0,4
2,azahara.garcia@originaltelecom.es,1.0,0.0,0.0,0.0,1,5


Sorted Pending Signature DataFrame:


...

,Inbound Telec.Orig.Sevilla,83,378,399,310,1170,email_value
0,albaaraujo@originaltelecom.es,0.0,0.0,10.0,8.0,18,1
1,albertocanto@originaltelecom.es,0.0,13.0,5.0,7.0,25,2
82,albertosanchez@originaltelecom.es,0.0,0.0,0.0,0.0,0,3
83,antonio.reina@originaltelecom.es,0.0,0.0,0.0,0.0,0,4
2,azahara.garcia@originaltelecom.es,12.0,0.0,0.0,0.0,12,5


Sorted Pending Signature DataFrame:


,Inbound Telec.Orig.Sevilla,111,465,515,360,1451,email_value
0,albaaraujo@originaltelecom.es,0.0,0.0,13.0,17.0,30,1
1,albertocanto@originaltelecom.es,0.0,11.0,10.0,7.0,28,2
71,albertosanchez@originaltelecom.es,0.0,0.0,0.0,0.0,0,3
72,antonio.reina@originaltelecom.es,0.0,0.0,0.0,0.0,0,4
2,azahara.garcia@originaltelecom.es,18.0,0.0,0.0,0.0,18,5


Sorted Pending Signature DataFrame:


,Inbound Telec.Orig.Sevilla,187,715,785,625,2312,email_value
0,albaaraujo@originaltelecom.es,0.0,0.0,25.0,20.0,45,1
1,albertocanto@originaltelecom.es,0.0,19.0,11.0,12.0,42,2
93,albertosanchez@originaltelecom.es,0.0,0.0,0.0,0.0,0,3
94,antonio.reina@originaltelecom.es,0.0,0.0,0.0,0.0,0,4
3,azahara.garcia@originaltelecom.es,29.0,0.0,0.0,0.0,29,5


## Remove 'email_value' Column

After sorting the DataFrames based on the email values, the 'email_value' column should be removed to prevent interference with further calculations.

In [9]:
# Function to remove the 'email_value' column
def remove_email_value_column(df):
    if 'email_value' in df.columns:
        df = df.drop(columns=['email_value'])
    return df

# Apply the function to each DataFrame
active_df = remove_email_value_column(active_df)
canceled_df = remove_email_value_column(canceled_df)
pending_signature_df = remove_email_value_column(pending_signature_df)
value_df = remove_email_value_column(value_df)
packs_df = remove_email_value_column(packs_df)
services_df = remove_email_value_column(services_df)


# Display the updated DataFrames without the 'email_value' column
print("Active DataFrame after removing 'email_value' column:")
display(active_df.head())

print("Canceled DataFrame after removing 'email_value' column:")
display(canceled_df.head())

print("Pending Signature DataFrame after removing 'email_value' column:")
display(pending_signature_df.head())

print("Value DataFrame after removing 'email_value' column:")
display(value_df.head())

print("Packs DataFrame after removing 'email_value' column:")
display(packs_df.head())

print("Services DataFrame after removing 'email_value' column:")
display(services_df.head())

column_values = active_df.iloc[:, 0].tolist()
print("Values from the second column of active_df:")
print(column_values)

Active DataFrame after removing 'email_value' column:


,Inbound Telec.Orig.Sevilla,187,715,785,625,2312
0,albaaraujo@originaltelecom.es,0.0,0.0,25.0,20.0,45
1,albertocanto@originaltelecom.es,0.0,19.0,11.0,12.0,42
93,albertosanchez@originaltelecom.es,0.0,0.0,0.0,0.0,0
94,antonio.reina@originaltelecom.es,0.0,0.0,0.0,0.0,0
3,azahara.garcia@originaltelecom.es,29.0,0.0,0.0,0.0,29


Canceled DataFrame after removing 'email_value' column:


,Inbound Telec.Orig.Sevilla,16,62,40,40.1,158
51,albaaraujo@originaltelecom.es,0.0,0.0,0.0,0.0,0
1,albertocanto@originaltelecom.es,0.0,1.0,0.0,1.0,2
52,albertosanchez@originaltelecom.es,0.0,0.0,0.0,0.0,0
53,antonio.reina@originaltelecom.es,0.0,0.0,0.0,0.0,0
2,azahara.garcia@originaltelecom.es,1.0,0.0,0.0,0.0,1


Pending Signature DataFrame after removing 'email_value' column:


...

Packs DataFrame after removing 'email_value' column:


,Inbound Telec.Orig.Sevilla,111,465,515,360,1451
0,albaaraujo@originaltelecom.es,0.0,0.0,13.0,17.0,30
1,albertocanto@originaltelecom.es,0.0,11.0,10.0,7.0,28
71,albertosanchez@originaltelecom.es,0.0,0.0,0.0,0.0,0
72,antonio.reina@originaltelecom.es,0.0,0.0,0.0,0.0,0
2,azahara.garcia@originaltelecom.es,18.0,0.0,0.0,0.0,18


Services DataFrame after removing 'email_value' column:


,Inbound Telec.Orig.Sevilla,187,715,785,625,2312
0,albaaraujo@originaltelecom.es,0.0,0.0,25.0,20.0,45
1,albertocanto@originaltelecom.es,0.0,19.0,11.0,12.0,42
93,albertosanchez@originaltelecom.es,0.0,0.0,0.0,0.0,0
94,antonio.reina@originaltelecom.es,0.0,0.0,0.0,0.0,0
3,azahara.garcia@originaltelecom.es,29.0,0.0,0.0,0.0,29


Values from the second column of active_df:
['albaaraujo@originaltelecom.es', 'albertocanto@originaltelecom.es', 'albertosanchez@originaltelecom.es', 'antonio.reina@originaltelecom.es', 'azahara.garcia@originaltelecom.es', 'beatriz.gomez@originaltelecom.es', 'maricarmen.cornejo@originaltelecom.es', 'carolinafuentes@originaltelecom.es', 'cesar.arnaldo@originaltelecom.es', 'david.molero@originaltelecom.es', 'elenaborrero@originaltelecom.es', 'estefania.panea@originaltelecom.es', 'francisco.perdomo@originaltelecom.es', 'gonzalofalcon@originaltelecom.es', 'guillermo.hurtado@originaltelecom.es', 'irati.izaguirre@originaltelecom.es', 'ivan.barroso@originaltelecom.es', 'laura.eguens@originaltelecom.es', 'lailasetati@originaltelecom.es', 'leonor.lopez@originaltelecom.es', 'dolores.cortes@originaltelecom.es', 'manuelcabra@originaltelecom.es', 'manuelvaldes@originaltelecom.es', 'manuelventura@originaltelecom.es', 'mariangeles.bueso@originaltelecom.es', 'mariaarroyo@originaltelecom.es', 'maria.to

## Process Data from all DataFrames

Sum all numeric values in each row, divide the result by 2, and store the final values.

In [10]:
# Calculate sum of numeric values in each row, divided by 2
active_sums = active_df.iloc[:, 1:].sum(axis=1) / 2
canceled_sums = canceled_df.iloc[:, 1:].sum(axis=1) / 2
pending_signature_sums = pending_signature_df.iloc[:, 1:].sum(axis=1) / 2
value_sums = value_df.iloc[:, 1:].sum(axis=1) / 2
packs_sums = packs_df.iloc[:, 1:].sum(axis=1) / 2
services_sums = services_df.iloc[:, 1:].sum(axis=1) / 2

print("Calculated sums for 'Active' DataFrame:")
print(active_sums.head(66))# Calculate sum of numeric values in each row, divided by 2 for 'Pending Signature' DataFrame
print("Calculated sums for 'Canceled' DataFrame:")
print(canceled_sums.head())
print("Calculated sums for 'Pending Signature' DataFrame:")
print(pending_signature_sums.head())
print("Calculated sums for 'Value' DataFrame:")
print(value_sums.head())
print("Calculated sums for 'Packs' DataFrame:")
print(packs_sums.head())
print("Calculated sums for 'Services' DataFrame:")
print(services_df.head())

Calculated sums for 'Active' DataFrame:
0      45.0
1      42.0
93      0.0
94      0.0
3      29.0
5      34.0
59     51.0
6      72.0
7      21.0
8      92.0
12     78.0
13     46.0
25     56.0
26     41.0
27    102.0
28     65.0
29     63.0
38     80.0
37     21.0
95      0.0
11     71.0
96      0.0
52     50.0
97      0.0
58     17.0
57     47.0
54     59.0
61     26.0
70     84.0
71     84.0
72     62.0
73     40.0
53    103.0
75     68.0
79     82.0
80     71.0
81     18.0
98      0.0
88     95.0
89     58.0
99      0.0
90     34.0
91     24.0
dtype: float64
Calculated sums for 'Canceled' DataFrame:
51    0.0
1     2.0
52    0.0
53    0.0
2     1.0
dtype: float64
Calculated sums for 'Pending Signature' DataFrame:
0     2.0
2     1.0
26    0.0
27    0.0
3     1.0
dtype: float64
Calculated sums for 'Value' DataFrame:
0     18.0
1     25.0
82     0.0
83     0.0
2     12.0
dtype: float64
Calculated sums for 'Packs' DataFrame:
0     30.0
1     28.0
71     0.0
72     0.0
2     18.0
dty

## Update finalFile Excel Sheet

Update the "finalFile" Excel sheet with the calculated values from all five DataFrame

### Update finalFile Excel sheet with Active Data

Update the "finalFile" Excel sheet with the calculated values from the Active DataFrame in column J.

In [11]:
from openpyxl import load_workbook

# Path to the final file
final_file_path ='/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_SEPTIEMBRE_24.xlsx'
sheet_name = 'GLOBAL SEPTIEMBRE'  # Change this to the correct sheet name

def update_final_file(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column F (6th column)
    start_row = 3
    column = 8  # Column H
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}'.")

# Update the final file with the calculated sums
update_final_file(final_file_path, sheet_name, active_sums)

Updated 43 rows in 'GLOBAL SEPTIEMBRE' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_SEPTIEMBRE_24.xlsx'.


### Update finalFile Excel Sheet with Canceled Data

Update the "finalFile" Excel sheet with the calculated values from the Canceled DataFrame in column K.

In [12]:
def update_final_file_canceled(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column N (14th column)
    start_row = 3
    column = 9  # Column I
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Canceled data.")

# Update the final file with the calculated sums for Canceled
update_final_file_canceled(final_file_path, sheet_name, canceled_sums)

Updated 43 rows in 'GLOBAL SEPTIEMBRE' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_SEPTIEMBRE_24.xlsx' with Canceled data.


### Update finalFile Excel Sheet with Pending Signature Data

Update the "finalFile" Excel sheet with the calculated values from the Pending Signature DataFrame in column L.

In [13]:
def update_final_file_pending_signature(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column Q (17th column)
    start_row = 3
    column = 10  # Column J
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Pending Signature data.")

# Update the final file with the calculated sums for Pending Signature
update_final_file_pending_signature(final_file_path, sheet_name, pending_signature_sums)

Updated 43 rows in 'GLOBAL SEPTIEMBRE' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_SEPTIEMBRE_24.xlsx' with Pending Signature data.


### Update finalFile Excel Sheet with Values Data

Update the "finalFile" Excel sheet with the calculated values from the Values DataFrame in column M.

In [14]:
def update_final_file_pending_signature(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column Q (17th column)
    start_row = 3
    column = 11  # Column K
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Values data.")

# Update the final file with the calculated sums for Pending Signature
update_final_file_pending_signature(final_file_path, sheet_name, value_sums)

Updated 43 rows in 'GLOBAL SEPTIEMBRE' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_SEPTIEMBRE_24.xlsx' with Values data.


### Update finalFile Excel Sheet with Packs Data

Update the "finalFile" Excel sheet with the calculated values from the Packs DataFrame in column N.

In [15]:
def update_final_file_pending_signature(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column Q (17th column)
    start_row = 3
    column = 12  # Column L
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Packs data.")

# Update the final file with the calculated sums for Pending Signature
update_final_file_pending_signature(final_file_path, sheet_name, packs_sums)

Updated 43 rows in 'GLOBAL SEPTIEMBRE' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_SEPTIEMBRE_24.xlsx' with Packs data.


### Update finalFile Excel Sheet with Services Data

Update the "finalFile" Excel sheet with the calculated values from the Services DataFrame in column O.

In [16]:
def update_final_file_pending_signature(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column Q (17th column)
    start_row = 3
    column = 13  # Column M
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Services data.")

# Update the final file with the calculated sums for Pending Signature
update_final_file_pending_signature(final_file_path, sheet_name, services_sums)

Updated 43 rows in 'GLOBAL SEPTIEMBRE' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_SEPTIEMBRE_24.xlsx' with Services data.
